# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")


## Create our QandA application

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [3]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [4]:
from langchain.embeddings import OpenAIEmbeddings


index = VectorstoreIndexCreator(
    embedding = OpenAIEmbeddings(),
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

C:\Users\luca5\AppData\Local\Temp\ipykernel_19696\2116028208.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(),
c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


**La chaîne RetrievalQA est dépreciée depuis peu, elle permettait la création de questions-réponses en langage naturel sur une source de données à l'aide de retrieval-augmented generation (RAG).**

In [5]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=deployment_name,
                        temperature=0.9
                        )

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [6]:
data[10]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [7]:
data[11]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

### Hard-coded examples

In [8]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [9]:
from langchain.evaluation.qa import QAGenerateChain


In [10]:
example_gen_chain = QAGenerateChain.from_llm(llm=llm)

In [11]:
# the warning below can be safely ignored

In [12]:
example_gen_chain

QAGenerateChain(verbose=False, prompt=PromptTemplate(input_variables=['doc'], input_types={}, partial_variables={}, template='You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\n{doc}\n<End Document>'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C640596E70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C640595BB0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x000001C640690E60>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001C640596EA0>, model_name='gpt-4o-assistant', temperature=0.9, model_kwargs={}, openai_api_key=SecretStr('**********'), disab

In [13]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\luca5\.conda\envs\GENAI\Lib\site-packages\langchain_openai\chat_models\base.py:358: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [14]:
new_examples[0]

{'qa_pairs': {'query': "What type of antimicrobial odor control is used in the Women's Campside Oxfords?",
  'answer': 'Cleansport NXT® antimicrobial odor control'}}

In [15]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

### Combine examples

In [16]:
examples += new_examples

In [17]:
qa.run(examples[0]["query"])

C:\Users\luca5\AppData\Local\Temp\ipykernel_19696\1223946598.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])




> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set has side pockets.'

## Manual Evaluation

In [18]:
import langchain
langchain.debug = True

In [19]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

'Yes, the Cozy Comfort Pullover Set has side pockets.'

In [20]:
# Turn off the debug mode
langchain.debug = False

## A votre tour avec la version non dépréciée

Créez une chaine de retrieval-QA avec LangChain, avec ou sans LCEL. Appliquez là aux `examples`, et testez là avec QAEvalChain comme dessous.

Indice: servez-vous de la documentation ! Notamment [ici](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html)

L'implémentation de LCEL expose les aspects internes de ce qui se passe autour de la récupération, du formatage des documents et de leur transmission au LLM à travers un prompt, mais elle est plus verbeuse. Vous pouvez personnaliser et envelopper cette logique de composition dans une fonction helper, ou utiliser la méthode helper de plus haut niveau `create_retrieval_chain` et `create_stuff_documents_chain`.

## LLM assisted evaluation

Original Notebook :

In [ ]:
predictions = qa.apply(examples)

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

In [ ]:
graded_outputs[0]

### A vous de jouer :

Ici on cherche à appliquer notre qa_chain à tous les examples, pour avoir une prédiction par exemple.

> Comment appliquer une chaine à une liste d'exemples ?

Une chaine est un objet `RunnableSequence`

Regarder la [documentation](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html) pour voir quelles méthodes on peut appeler sur une chaine.

In [41]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'qa_pairs': {'query': "What type of material is used for the construction of the Women's Campside Oxfords, and what is the purpose of the Cleansport NXT® feature included in the innersole?",
   'answer': "The Women's Campside Oxfords are constructed with soft canvas material. The purpose of the Cleansport NXT® feature included in the innersole is to provide antimicrobial odor control."}},
 {'qa_pairs': {'query': 'What are the dimensions of the medium-sized Recycled Waterhog Dog Mat?',
   'answer': 'The dimensions of the medium-sized Recycled Waterhog Dog Mat are 22.5" x 34.5".'}},
 {'qa_pairs': {'query': "What are the key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
   'answer': "The key features of the Infant and Toddler Girls'

## [LangSmith Platform](https://smith.langchain.com/)

What is Langsmith? What can you do with it ?